<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
# Import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Carla_Ferreira_API_challenge').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

Declare the directory to write the data that came from API carrismetropolitana

In [9]:
!mkdir -p /content/lake/bronze

In [3]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [7]:
#Schemas
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)
                             ])


lines_schema = StructType([StructField('_corrupt_record', StringType(), True),
                           StructField('color', StringType(), True),
                           StructField('facilities', StringType(), True),
                           StructField('id', StringType(), True),
                           StructField('localities', StringType(), True),
                           StructField('long_name', StringType(), True),
                           StructField('municipalities', StringType(), True),
                           StructField('patterns', StringType(), True),
                           StructField('routes', StringType(), True),
                           StructField('short_name', StringType(), True),
                           StructField('text_color', StringType(), True)
                           ])

municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                    StructField('district_name', StringType(), True),
                                    StructField('id', StringType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('prefix', StringType(), True),
                                    StructField('region_id', StringType(), True),
                                    StructField('region_name', StringType(), True)
                                    ])

In [8]:
vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", schema = vehicle_schema)
lines = readFromAPI("https://api.carrismetropolitana.pt/lines", schema = lines_schema)
municipalities = readFromAPI("https://api.carrismetropolitana.pt/municipalities", schema = municipalities_schema)

In [16]:
# create "date" extracted from "timestamp" column (format: hh24miss)
from pyspark.sql.functions import date_format
vehicles = vehicles.withColumn("date", date_format('timestamp', "HHmmss"))

vehicles.show()

+-------+--------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+------+
|bearing|            block_id|current_status|      id|      lat|line_id|       lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|  date|
+-------+--------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+------+
|    333|20241121-64010132...|   INCOMING_AT|44|12060|38.715164|   4504| -8.902005|  4504_0_1|  4504_0|            SCHEDULED|121150234560|     15.0| 010126|2024-11-21 15:05:27|4504_0_1|2100|143...|150527|
|     32|20241121-64010250...|   INCOMING_AT|44|12089| 38.53214|   4421| -8.876826|  4421_0_2|  4421_0|            SCHEDULED|112020234560| 8.055555| 160432|2024-11-21 15:05:17|4421

In [ ]:
#df = extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")